<a href="https://colab.research.google.com/github/Sandeep5500/kaggle_attempts/blob/main/Titanic_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**0.77751** accuracy with **XGBoost** model on Titanic dataset.
(Random forests attempt included as well)



In [1]:
## Kaggle conf
!pip install kaggle
!mkdir .kaggle
import json
token = {"username":"sandeepkumar00","key":"006083ef889a4c38471af6b2ce139874"}
with open('.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)  

mkdir: cannot create directory ‘.kaggle’: File exists


In [2]:
!cp .kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v .
!chmod 600 .kaggle/kaggle.json
!kaggle datasets list
!kaggle competitions download -c titanic 

- path is now set to: .
ref                                                            title                                               size  lastUpdated          downloadCount  
-------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  
arashnic/hr-analytics-job-change-of-data-scientists            HR Analytics: Job Change of Data Scientists        295KB  2020-12-07 00:25:10            596  
babyoda/access-to-computers-from-home-oecd                     Access to Computers From Home OECD                   3KB  2020-12-04 10:50:09             13  
shashwatwork/impact-of-covid19-pandemic-on-the-global-economy  Impact of Covid-19 Pandemic on the Global Economy    1MB  2020-11-29 14:16:30            141  
emmanuelleai/world-marathons-majors                            World Marathons Majors                               7KB  2020-12-06 19:24:28             21  
mrmorj/dataset-of-songs-in-s

In [3]:
import pandas as pd
import numpy as np

train = pd.read_csv('/content/competitions/titanic/train.csv', header=0)
test = pd.read_csv('/content/competitions/titanic/test.csv', header=0)

In [54]:
print(train)
print(test)
final_train_y = train["Survived"]

     PassengerId  Survived  Pclass  ... Cabin Embarked  Married Woman
0              1         0       3  ...   NaN        S              0
1              2         1       1  ...   C85        C              1
2              3         1       3  ...   NaN        S              0
3              4         1       1  ...  C123        S              1
4              5         0       3  ...   NaN        S              0
..           ...       ...     ...  ...   ...      ...            ...
886          887         0       2  ...   NaN        S              0
887          888         1       1  ...   B42        S              0
888          889         0       3  ...   NaN        S              1
889          890         1       1  ...  C148        C              0
890          891         0       3  ...   NaN        Q              0

[891 rows x 13 columns]
     PassengerId  Pclass  ... Embarked Married Woman
0            892       3  ...        Q             0
1            893       3  ...

In [56]:
train["Married Woman"] = np.where(train["Name"].str.len() >30, 1, 0)
test["Married Woman"] = np.where(test["Name"].str.len() >30, 1, 0)

In [57]:
cols_of_interest1 = ["Pclass","Sex", "Age", "Embarked","Fare","Married Woman"]
cols_of_interest2 = ["Pclass","Sex","Age","Embarked","Fare","Married Woman"]
cor_train = train.loc[:,cols_of_interest1]
print(cor_train)
cor_test = test.loc[:,cols_of_interest2]
print(cor_test)

total = cor_train.append(cor_test)
print(total)

     Pclass     Sex   Age Embarked     Fare  Married Woman
0         3    male  22.0        S   7.2500              0
1         1  female  38.0        C  71.2833              1
2         3  female  26.0        S   7.9250              0
3         1  female  35.0        S  53.1000              1
4         3    male  35.0        S   8.0500              0
..      ...     ...   ...      ...      ...            ...
886       2    male  27.0        S  13.0000              0
887       1  female  19.0        S  30.0000              0
888       3  female   NaN        S  23.4500              1
889       1    male  26.0        C  30.0000              0
890       3    male  32.0        Q   7.7500              0

[891 rows x 6 columns]
     Pclass     Sex   Age Embarked      Fare  Married Woman
0         3    male  34.5        Q    7.8292              0
1         3  female  47.0        S    7.0000              1
2         2    male  62.0        Q    9.6875              0
3         3    male  27.0   

In [58]:
total.isnull().sum()

Pclass             0
Sex                0
Age              263
Embarked           2
Fare               1
Married Woman      0
dtype: int64

In [59]:
total_imp = total

avg_age = total['Age'].mean()
total_imp["Age"].fillna(avg_age,inplace=True)
total_imp["Fare"].fillna(total['Fare'].mean(),inplace=True)

mode_cols = ["Embarked"]
total_imp[mode_cols]=total_imp[mode_cols].fillna(total_imp.mode().iloc[0])


In [61]:
total.isnull().sum()

Pclass           0
Sex              0
Age              0
Embarked         0
Fare             0
Married Woman    0
dtype: int64

In [62]:
onehot_cols = ["Pclass","Sex", "Embarked"]
print(onehot_cols)
total_oh_df = pd.get_dummies(total_imp, columns=onehot_cols)
print(total_oh_df)
print(total_oh_df.columns)

['Pclass', 'Sex', 'Embarked']
           Age      Fare  Married Woman  ...  Embarked_C  Embarked_Q  Embarked_S
0    22.000000    7.2500              0  ...           0           0           1
1    38.000000   71.2833              1  ...           1           0           0
2    26.000000    7.9250              0  ...           0           0           1
3    35.000000   53.1000              1  ...           0           0           1
4    35.000000    8.0500              0  ...           0           0           1
..         ...       ...            ...  ...         ...         ...         ...
413  29.881138    8.0500              0  ...           0           0           1
414  39.000000  108.9000              0  ...           1           0           0
415  38.500000    7.2500              0  ...           0           0           1
416  29.881138    8.0500              0  ...           0           0           1
417  29.881138   22.3583              0  ...           1           0           

In [63]:
final_train_x = total_oh_df.iloc[:891]
final_test_x = total_oh_df.iloc[891:]
print(final_train_x)
print(final_test_x)

           Age     Fare  Married Woman  ...  Embarked_C  Embarked_Q  Embarked_S
0    22.000000   7.2500              0  ...           0           0           1
1    38.000000  71.2833              1  ...           1           0           0
2    26.000000   7.9250              0  ...           0           0           1
3    35.000000  53.1000              1  ...           0           0           1
4    35.000000   8.0500              0  ...           0           0           1
..         ...      ...            ...  ...         ...         ...         ...
886  27.000000  13.0000              0  ...           0           0           1
887  19.000000  30.0000              0  ...           0           0           1
888  29.881138  23.4500              1  ...           0           0           1
889  26.000000  30.0000              0  ...           1           0           0
890  32.000000   7.7500              0  ...           0           1           0

[891 rows x 11 columns]
           Age 

In [64]:
import xgboost as xgb

In [65]:
gbm = xgb.XGBClassifier(objective="binary:logistic", random_state=0, learning_rate=0.05, max_depth = 9, n_estimators= 500, reg_alpha= 0.5, reg_lambda= 0.5,colsample_bytree=0.4)

gbm.fit(final_train_x, final_train_y)
predictions = gbm.predict(final_test_x)

# Dump predictions into csv file
submission = pd.DataFrame({ 'PassengerId': test["PassengerId"],
                            'Survived': predictions })
submission.to_csv("submission.csv", index=False)


**Submission Comparer**

In [52]:
sub1 = pd.read_csv('/content/submission4.csv', header=0)
sub2 = pd.read_csv('/content/submission.csv', header=0)
n_diff = 0
for ind in sub1.index: 
     if sub1['Survived'][ind] != sub2['Survived'][ind]:
         n_diff+=1
         print(sub1['PassengerId'][ind])
print("Total Diff: ",n_diff)

898
910
926
927
946
964
965
982
986
1010
1028
1049
1051
1063
1098
1141
1198
1205
1210
1261
1271
1274
1309
Total Diff:  23


In [151]:
import sklearn
from sklearn.model_selection import GridSearchCV

gbm2 = xgb.XGBClassifier(objective="binary:logistic", random_state=0, learning_rate = 0.05)
params = {
        "n_estimators": [500, 700, 1000],
        "max_depth": [ 7, 9, 11],
        "reg_alpha": [0.3, 0.5, 0.7],
        "reg_lambda": [0.3, 0.5, 0.8],
        "colsample_bytree": [0.4, 0.7, 1]
    }
grid = GridSearchCV(gbm2, params, refit = True, verbose = 1,cv =4, n_jobs=-1) 
   
# fitting the model for grid search 
grid.fit(final_train_x, final_train_y) 
 
# print best parameter after tuning 
print(grid.best_params_) 


Fitting 4 folds for each of 243 candidates, totalling 972 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   58.2s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 972 out of 972 | elapsed:  6.3min finished


{'colsample_bytree': 0.4, 'max_depth': 9, 'n_estimators': 500, 'reg_alpha': 0.5, 'reg_lambda': 0.5}


In [156]:
print(grid.best_score_)
grid_predictions = grid.predict(final_test_x) 
# Dump predictions into csv file
submission = pd.DataFrame({ 'PassengerId': test["PassengerId"],
                            'Survived': grid_predictions })
submission.to_csv("submission2.csv", index=False)

0.8361410738092352


**Random Forests**

In [41]:
from sklearn.ensemble import RandomForestClassifier

regmodel = RandomForestClassifier(n_estimators=20, random_state=0)
regmodel.fit(final_train_x, final_train_y)
regpredictions = regmodel.predict(final_test_x)

In [66]:
# Dump predictions into csv file
submission = pd.DataFrame({ 'PassengerId': test["PassengerId"],
                            'Survived': regpredictions })
submission.to_csv("submission3.csv", index=False)

In [68]:
from sklearn.model_selection import RandomizedSearchCV
rf = RandomForestClassifier( random_state=0)
random_grid = {
'bootstrap': [True, False],
 'max_depth': [10, 40, 70, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 600, 800, 1400]}

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, n_jobs = -1)
# Fit the random search model
rf_random.fit(final_train_x, final_train_y)
regrandpredictions = rf_random.predict(final_test_x)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   28.9s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  4.5min finished


In [70]:
print(rf_random.best_score_)
print(rf_random.best_params_)
submission = pd.DataFrame({ 'PassengerId': test["PassengerId"],
                            'Survived': regrandpredictions })
submission.to_csv("submission4.csv", index=False)

0.8271604938271605
{'n_estimators': 600, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'auto', 'max_depth': 10, 'bootstrap': False}
